In [ ]:
import pandas as pd
# create dataframe from Coding Template
coding_template_df = pd.read_csv('/content/IS262A_Coding_Template.csv')
# create list from Province/Municipality column
provincemunicipality_list = coding_template_df['Province/Municipality'].tolist()
print(provincemunicipality_list)
# separate provinces/autonomous regions by ;
provinces_list = []
for item in provincemunicipality_list:
    if isinstance(item, str):
        provinces_list.extend([s.strip() for s in item.split(';')])
print(provinces_list)

[nan, nan, nan, 'Qinghai Province; Sichuan Province; Gansu Province; Ningxia Hui Autonomous Region; Inner Mongolia Autonomous Region; Shaanxi Province; Shanxi Province; Henan Province; Shandong Province', 'Ningxia Hui Autonomous Region', nan, 'Shandong Province', nan, nan, nan, nan, 'Henan Province', nan, 'Shanxi Province; Henan Province', nan, nan, nan, nan, 'Henan Province; Shandong Province; Jiangsu Province; Zhejiang Province; Hunan Province; Hubei Province; Shaanxi Province; Shanxi Province', nan, 'Liaoning Province', nan, nan, nan, nan, nan, 'Jiangsu Province', nan, 'Anhui Province', 'Shaanxi Province; Shanxi Province', 'Fujian Province; Jiangxi Province; Hunan Province', 'Shaanxi Province', nan, 'Jiangsu Province', nan, "Hebei Province; Beijing Municipality; (what's listed for specific study, but Great Wall spans 16 provinces)", 'Chongqing Municipality; Sichuan Province', 'Shandong Province', nan, nan, 'Yunnan Province', 'Hunan Province', nan, nan, nan, nan, nan, nan, nan, nan, 

In [ ]:
import geopandas as gpd
# create dataframe from GADM China administrative subdivisions 1 (provinces)
gadm_chn_1_df = gpd.read_file('/gadm41_CHN_1.shp')
print(gadm_chn_1_df)

## prepare provinces_list for table join
# remove suffixes to reflect GADM 'NAME_1' province names only
def removeSuffixs(sentence):
    if not isinstance(sentence, str):
        return sentence

    suffixList = [" Province", " Autonomous Region", " Municipality"]
    modified_sentence = sentence

    for suffix in suffixList:
        if modified_sentence.endswith(suffix):
            modified_sentence = modified_sentence[:-len(suffix)].strip()
            break

    return modified_sentence

cleaned_provinces_list = [removeSuffixs(p) for p in provinces_list]

# create attribute table from cleaned provinces
provinces_df = pd.DataFrame({"NAME_1": cleaned_provinces_list})

# match by attribute
matched_gdf = provinces_df.merge(
    gadm_chn_1_df[["NAME_1", "geometry"]],
    on="NAME_1",
    how="left")

# convert to GeoDataFrame
matched_gdf = gpd.GeoDataFrame(matched_gdf, geometry="geometry")
print(matched_gdf)

# count province occurrences
province_counts = (
    pd.Series(cleaned_provinces_list)
      .value_counts()
      .rename_axis("NAME_1")
      .reset_index(name="count"))

print(province_counts)

# join counts onto GADM GeoDataFrame
gadm_with_counts = gadm_chn_1_df.merge(
    province_counts,
    on="NAME_1",
    how="left"
)

# replace NaN with zero (province not mentioned)
gadm_with_counts["count"] = gadm_with_counts["count"].fillna(0).astype(int)

# Export to GeoJSON
gadm_province_total.to_file("gadm_provincetotal.geojson", driver="GeoJSON")

DataSourceError: /gadm41_CHN_1.shp: No such file or directory